In [1]:
import pandas as pd
import numpy as np

hitdata = pd.read_csv("2024_batproj_clean.csv") # Read in fangraphs ATC hitter proj
hitdata = hitdata.drop(['ADP'], axis=1)

fp_data = pd.read_csv("fp_hit.csv")             # Read in Fantasypros data (for positions)
fp_data["Name"] = fp_data["Player"]
fp_data = fp_data[["Name", "Positions", "Rank", "ADP"]]
fp_data['Positions'] = fp_data['Positions'].str.replace('LF','OF')
fp_data['Positions'] = fp_data['Positions'].str.replace('CF','OF')
fp_data['Positions'] = fp_data['Positions'].str.replace('RF','OF')

# Remove dupes
s=fp_data.Positions.str.get_dummies(',')
fp_data['Positions']=s.dot(s.columns+',').str[:-1]

In [2]:
fp_data[fp_data['Positions'].str.contains('OF')].index[0:12]

Int64Index([0, 2, 3, 4, 5, 6, 8, 10, 13, 23, 25, 29], dtype='int64')

In [3]:
hitdata = hitdata.merge(fp_data, on="Name", how="left")

hitdata["Pts"] = hitdata["R"] + hitdata["HR"] + hitdata["RBI"] + hitdata["SB"] + hitdata["BB"] - 0.5*hitdata["SO"] + hitdata["H"] + hitdata["2B"] + 2*hitdata["3B"] + 3*hitdata["HR"]
hitdata["PPG"] =  hitdata["Pts"] / hitdata["G"]
hitdata = hitdata.dropna()
hitdata = hitdata[hitdata["G"] > 80]
hitdata = hitdata.sort_values(by=['Pts'], ascending=False)
hitdata = hitdata.reset_index(drop=True)

In [4]:
roster = ["C", "2B", "SS", "3B", "1B", "OF", "OF", "OF", "2B|SS", "1B|3B", "C|1B|2B|3B|SS|OF", "C|1B|2B|3B|SS|OF", "C|1B|2B|3B|SS|OF"]

hit_rp = hitdata

# remove all the starters
for r in roster:
    first12 = hit_rp[hit_rp['Positions'].str.contains(r)].index[0:12] # Get the first 12 in the list
        
    #print(first12)
    
    hit_rp = hit_rp.drop(hit_rp.index[first12]) # remove the first 12
    hit_rp = hit_rp.reset_index(drop=True)

# Calculate average of the top 5 players at each position
ps = ["C", "1B", "2B", "3B", "SS", "OF"]
rpv = pd.DataFrame(columns = ['Position', 'RPV'])
for p in ps:
    rp5 = hit_rp[hit_rp['Positions'].str.contains(p)][0:5] # get the first five from the replacement player list
    #print(rp5)
    rpv = rpv.append({'Position' : p, 'RPV' : np.mean(rp5["PPG"])}, ignore_index = True)

In [5]:
rpv = rpv.sort_values(by=['RPV'], ascending=False)
rpv

,Position,RPV
5,OF,2.671214
1,1B,2.621560
2,2B,2.533906
3,3B,2.525174
4,SS,2.499602
0,C,2.464405


In [6]:
hitdata["VORPPG"] = 0
for index, row in rpv.iterrows():
    
    hitdata.loc[hitdata['Positions'].str.contains(row["Position"]), ["VORPPG"]] = hitdata["PPG"] - row["RPV"]
hitdata["aVORPPG"] = hitdata["VORPPG"] * hitdata["G"] / 162
hitdata["aVORPPW"] = hitdata["aVORPPG"] * 5.5
hitdata["VORPPW"] = hitdata["VORPPG"] * 5.5

In [7]:
hitdata = hitdata.sort_values(by=['aVORPPW'], ascending=False)
# hitdata = hitdata.reset_index(drop=True)
# hitdata["HRank"] = hitdata.index + 1
# hitdata["H Score"] = hitdata["Rank"] - hitdata["HRank"]

In [8]:
hitdata.to_csv("hrank_hit24.csv", index=False)
hitdata

,#,Name,Team,G,PA,AB,H,2B,3B,HR,...,IntraSD,Positions,Rank,ADP,Pts,PPG,VORPPG,aVORPPG,aVORPPW,VORPPW
0,1,Ronald Acuna Jr.,ATL,147,659,566,177,33,2,35,...,1.16,OF,1.0,1.0,644.0,4.380952,1.709739,1.551430,8.532862,9.403563
2,2,Mookie Betts,LAD,149,661,566,160,35,2,33,...,0.87,"2B,OF,SS",4.0,3.0,578.0,3.879195,1.379592,1.268884,6.978864,7.587759
1,4,Juan Soto,NYY,153,664,529,148,27,2,37,...,0.96,OF,9.0,8.0,614.0,4.013072,1.341858,1.267310,6.970208,7.380220
7,3,Aaron Judge,NYY,138,600,496,137,22,0,44,...,1.41,OF,11.0,10.0,562.0,4.072464,1.401250,1.193657,6.565116,7.706875
6,5,Yordan Alvarez,HOU,139,599,505,150,31,2,39,...,1.49,OF,14.0,16.0,563.5,4.053957,1.382743,1.186428,6.525352,7.605087
4,20,Matt Olson,ATL,153,663,564,151,28,1,40,...,1.65,1B,13.0,13.0,569.0,3.718954,1.097394,1.036428,5.700352,6.035667
8,7,Jose Ramirez,CLE,151,657,577,160,37,4,26,...,0.32,3B,15.0,14.0,549.0,3.635762,1.110588,1.035177,5.693475,6.108232
3,15,Kyle Tucker,HOU,151,644,565,159,34,4,31,...,0.36,OF,6.0,6.0,570.0,3.774834,1.103621,1.028683,5.657759,6.069914
5,9,Freddie Freeman,LAD,154,677,587,180,39,2,26,...,1.17,1B,8.0,7.0,567.0,3.681818,1.060258,1.007900,5.543447,5.831419
9,12,Bobby Witt Jr.,KCR,152,657,604,169,33,7,29,...,0.90,SS,2.0,4.0,542.5,3.569079,1.069477,1.003460,5.519028,5.882122


In [9]:
len("AS")

2

In [10]:
pitdata = pd.read_csv("2024_pitproj.csv") # Read in fangraphs ATC hitter proj
pitdata = pitdata.drop(['ADP'], axis=1)

fp_pit = pd.read_csv("fp_pit.csv")             # Read in Fantasypros data (for positions)
fp_pit["Name"] = fp_pit["Player"]
fp_pit = fp_pit[["Name", "Positions", "Rank", "ADP"]]
fp_pit['Positions'] = fp_pit['Positions'].str.replace('LF','OF')
fp_pit['Positions'] = fp_pit['Positions'].str.replace('CF','OF')
fp_pit['Positions'] = fp_pit['Positions'].str.replace('RF','OF')

# Remove dupes
s=fp_pit.Positions.str.get_dummies(',')
fp_pit['Positions']=s.dot(s.columns+',').str[:-1]

In [11]:
pitdata["Team"] = np.where(pitdata["Team"].isna(), "FA", pitdata["Team"])

In [12]:
pitdata = pitdata.merge(fp_pit, on="Name", how="left")

In [13]:
pitdata["Pts"] = 3*pitdata["IP"] + 5*pitdata["W"] - 5*pitdata["L"] + 5*pitdata["SV"] - pitdata["H"] - 2*pitdata["ER"] - pitdata["BB"] + pitdata["SO"]
pitdata["PPG"] =  pitdata["Pts"] / pitdata["G"]
pitdata.to_csv("pitdata_tmp.csv", index=False)

pitdata = pitdata.dropna()
pitdata = pitdata.sort_values(by=['Pts'], ascending=False)
pitdata = pitdata.reset_index(drop=True)

# Reclassify SP and RP based on GS (GS = G | GS > 10 == SP)
pitdata["Positions"] = np.where((pitdata['GS'] == pitdata['G']) | (pitdata['GS'] >= 10), "SP", "RP")
sp_df = pitdata[pitdata["Positions"] == "SP"]
rp_df = pitdata[pitdata["Positions"] == "RP"]

In [22]:
sp_df["VORPPG"] = sp_df["PPG"] - 9.5
sp_df["aVORPPG"] = sp_df["VORPPG"] * sp_df["GS"] / 31
sp_df["VORPPW"] = sp_df["VORPPG"] * 1.1
sp_df["aVORPPW"] = sp_df["aVORPPG"] * 1.1
sp_df = sp_df.reset_index(drop=True)
hitdata["HRank"] = hitdata.index + 1
hitdata["H Score"] = hitdata["Rank"] - hitdata["HRank"]

In [23]:
rp_df["VORPPG"] = rp_df["PPG"]
rp_df["aVORPPG"] = rp_df["VORPPG"]
rp_df["VORPPW"] = rp_df["VORPPG"] * 2.5
rp_df["aVORPPW"] = rp_df["aVORPPG"] * (rp_df["G"] / 162) * 5.5 - 9.5
rp_df = rp_df.sort_values(by=['aVORPPW'], ascending=False)

In [24]:
# Grab the same columns from the three dataframes (hitters, SP, RP)
RP1_df = rp_df[["Name", "Team", "G", "Positions","Rank", "ADP", "Pts", "PPG", "VORPPG", "VORPPW", "aVORPPG", "aVORPPW"]]
SP1_df = sp_df[["Name", "Team", "G", "Positions","Rank", "ADP", "Pts", "PPG", "VORPPG", "VORPPW", "aVORPPG", "aVORPPW"]]
HIT1_df = hitdata[["Name", "Team", "G", "Positions","Rank", "ADP", "Pts", "PPG", "VORPPG", "VORPPW", "aVORPPG", "aVORPPW"]]
proj_df = pd.concat([RP1_df, SP1_df, HIT1_df])
proj_df = proj_df.sort_values(by=['aVORPPW'], ascending=False)
proj_df = proj_df.reset_index(drop=True)
proj_df["HDP"] = proj_df.index + 1
proj_df["H"] = proj_df["ADP"] - proj_df["HDP"]
proj_df["eR"] = np.ceil(proj_df["ADP"]/12)
proj_df["hR"] = np.ceil(proj_df["HDP"]/12)
proj_df.rename(columns={'Positions':'Pos'}, inplace=True)

In [25]:
proj_df = proj_df[["HDP", "hR", "ADP", "eR", "H", "Pos", "Name", "Team", "G", "PPG", "aVORPPW"]]
proj_df.to_csv("hank_proj_24.csv", index=False)

In [26]:
np.ceil(4.0)


4.0

In [27]:
# Position Rankings
pos = ["C", "1B", "2B", "3B", "SS", "OF", "SP", "RP"]
for p in pos:
    pos_tmp = proj_df[proj_df['Pos'].str.contains(p)]
    pos_tmp.to_csv("2024_proj_" + p + ".csv", index=False)

In [28]:
# Round by Round Targets
targets_df = proj_df.sort_values(by=['eR', 'hR', 'HDP'], ascending=True)
targets_df = targets_df[targets_df["hR"] <= targets_df["eR"]]
hit_targets_df = targets_df[~targets_df['Pos'].str.contains("P")]
pit_targets_df = targets_df[targets_df['Pos'].str.contains("P")]

In [30]:
hit_targets_df.to_csv("HitterTargets_byRound.csv", index=False)
pit_targets_df.to_csv("PitcherTargets_byRound.csv", index=False)